In [99]:
from database.pymysql_conn import DataBase
from util.text import clean_text

import pandas as pd
import numpy as np
import re
import nltk
import pickle

from collections import Counter
from pathlib import Path
from nltk.corpus import stopwords
from scipy.stats import entropy

In [5]:
db = DataBase()

In [11]:
df_success = db.to_df("""
SELECT * FROM yt_comment where appid in (SELECT appid FROM get_new_games where avg_player_count > 1000) and filter=0 and language='en';
""")

In [12]:
df_fail = db.to_df("""
SELECT * FROM yt_comment where appid in (SELECT appid FROM get_new_games where avg_player_count < 1000) and filter=0 and language='en';
""")

In [165]:
df_games = {}

In [178]:
df_all = pd.concat([df_success, df_fail])

In [181]:
text_all = {}
for gameName in df_all.gameName.unique():
    raw = df_all[df_all['gameName']==gameName]['text'].tolist()
    text = ' '.join([clean_text(t) for t in raw])
    text_all[gameName] = text

In [192]:
stw = stopwords.words('english')
token_all = {}
for key, val in text_all.items():
    tok = nltk.word_tokenize(val)
    token_all[key] = [w for w in tok if not w in stw]

In [197]:
token_cnt_all = {}
for key, val in token_all.items():
    token_cnt_all[key] = {k: v for k, v in sorted(Counter(token_all[key]).items(), key=lambda item: item[1], reverse=True)}

## filtering

In [87]:
# likeCount 1개 이상인 경우만 필터링
df_success = df_success[df_success['likeCount'] > 1]
df_fail = df_fail[df_fail['likeCount'] > 1]

In [125]:
# outlier
# 732690 : FIVE NIGHTS AT FREDDYS VR HELP WANTED
df_fail = df_fail.drop(df_fail[df_fail['appid']==732690].index)

## tokenizing

In [126]:
raw_text_success = df_success['text'].tolist()

In [127]:
text_success = ' '.join([clean_text(t) for t in raw_text_success])

In [128]:
raw_text_fail = df_fail['text'].tolist()

In [129]:
text_fail = ' '.join([clean_text(t) for t in raw_text_fail])

In [212]:
stw = stopwords.words('english')

token_success = nltk.word_tokenize(text_success)
token_success = [w for w in token_success if not w in stw]

token_fail = nltk.word_tokenize(text_fail)
token_fail = [w for w in token_fail if not w in stw]

In [213]:
token_cnt_success = {k: v for k, v in sorted(Counter(token_success).items(), key=lambda item: item[1], reverse=True)}
token_cnt_fail = {k: v for k, v in sorted(Counter(token_fail).items(), key=lambda item: item[1], reverse=True)}

## entropy

In [132]:
word_list = list(set().union(token_success.keys(), token_fail.keys()))

In [214]:
entropy_list = {}
for word in word_list:
    s = token_cnt_success.get(word, 0)
    f = token_cnt_fail.get(word, 0)
    if s >50 or f>50:
        if entropy([s, f], base=2) > 0.0:
            entropy_list[word] = entropy([s, f], base=2)

In [215]:
{k: v for k, v in sorted(entropy_list.items(), key=lambda item: item[1], reverse=False)}

{'yakuza': 0.01537092083878113,
 'furry': 0.024581173073578405,
 'sean': 0.02911668108015386,
 'anno': 0.0310875616769587,
 'nico': 0.03524753954071269,
 'kiwami': 0.03853718088787195,
 'dunkey': 0.04022215086538087,
 'jacksepticeye': 0.05591963669874304,
 'chess': 0.07201647168541384,
 'cyrus': 0.0730661193138456,
 'conan': 0.07885601377455284,
 'ezio': 0.08013604733127526,
 'rtx': 0.08029923063040913,
 'ac3': 0.08248822045351054,
 'ironsight': 0.08354288831032232,
 'lily': 0.08574268253550253,
 'exclusives': 0.10802319526362833,
 'creed': 0.11241733891193453,
 'octopath': 0.1198929467195168,
 'layers': 0.12394986771100759,
 'morty': 0.13035034910142976,
 'scares': 0.13303964861069892,
 'unity': 0.1350362028021276,
 'rage': 0.13680050934270283,
 'elite': 0.14295535512672705,
 'dota': 0.15109697051711368,
 'ac': 0.15743149388734753,
 'justin': 0.1608230208224694,
 'sega': 0.1623261801753929,
 'horror': 0.1773894531859974,
 'fear': 0.17925606692832147,
 '1800': 0.17925606692832147,
 'bo

In [160]:
def get_entropy(word, token_a, token_b):
    cnt_a, cnt_b = token_a.get(word,0), token_b.get(word,0)
    ent = entropy([cnt_a, cnt_b], base=2)
    return cnt_a, cnt_b, ent

In [204]:
def get_count(word, token_cnt):
    for key, val in token_cnt.items():
        if word in val:
            print(key, val[word])

In [230]:
get_entropy("spider", token_cnt_success, token_cnt_fail)

(5, 71, 0.35001059292688497)

In [231]:
get_count("spider", token_cnt_all)

Yakuza Kiwami 2 1
RAGE 2 3
OCTOPATH TRAVELER 1
Paper Dolls Original 1
Blood Fresh Supply 2
Layers of Fear 2 2
Conan Unconquered 1
SpiderMan Far From Home Virtual Reality 65
